In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
# from nb_004c import *
from nb_005 import *

## Basic data aug

In [ ]:
PATH = Path('data/stl10')

In [ ]:
data_mean, data_std = map(tensor, ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))
data_norm,data_denorm = normalize_funcs(data_mean,data_std)

In [ ]:
train_ds = FilesDataset.from_folder(PATH/'train')
valid_ds = FilesDataset.from_folder(PATH/'valid')

In [ ]:
# 14 epochs -> 96.6

In [ ]:
size=96

tfms = get_transforms(do_flip=True)
tds = transform_datasets(train_ds, valid_ds, tfms, size=size)
data = DataBunch(*tds, bs=64, num_workers=8, tfms=data_norm)

## Train

In [ ]:
from torchvision.models import resnet18, resnet34, resnet50
arch = resnet50

In [ ]:
bn_types = (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)

def set_bn_eval(m):
    for l in m.children():
        set_bn_eval(l)
        if isinstance(l, bn_types) and not next(l.parameters()).requires_grad:
            l.momentum=0.0
            
@dataclass
class BnFreeze(Callback):
    learn:Learner
    def on_train_begin(self, **kwargs): set_bn_eval(self.learn.model)

In [ ]:
learn = ConvLearner(data, arch, 2, wd=1e-2, callback_fns=[BnFreeze])
learn.metrics = [accuracy]
learn.split(lambda m: (m[0][6], m[1]))
learn.freeze()

In [ ]:
lr = 1e-3

In [ ]:
learn.fit(1, lr)

In [ ]:
learn.save('0')

In [ ]:
def requires_grad(l):
    p = list(l.parameters())
    if not p: return None
    return p[0].requires_grad

In [ ]:
[requires_grad(o) for o in flatten_model(learn.model)]

## Gradual unfreezing

In [ ]:
learn = ConvLearner(data, arch, 2, wd=1e-2)
learn.metrics = [accuracy]

In [ ]:
learn.load('0')

In [ ]:
learn.split(lambda m: (m[0][6], m[1]))

In [ ]:
learn.freeze_to(1)

In [ ]:
lrs = np.array([lr/9, lr/3, lr])

In [ ]:
fit_one_cycle(learn, lrs, 3)

In [ ]:
learn.save('1')

In [ ]:
learn.load('1')

In [ ]:
learn.unfreeze()

In [ ]:
fit_one_cycle(learn, lrs/10, 3)

In [ ]:
fit_one_cycle(learn, lrs/10, 3)

In [ ]:
fit_one_cycle(learn, lrs/100, 3)

In [ ]:
learn.save('2')

In [ ]:
# TODO remove layer groups; use start_layer / start_lr

## TTA

In [ ]:
def transform_datasets(train_ds, valid_ds, tfms, size=None):
    return (DatasetTfm(train_ds, tfms[0], size=size),
            DatasetTfm(valid_ds, tfms[1], size=size),
            DatasetTfm(valid_ds, tfms[0], size=size))

In [ ]:
class DataBunch():
    def __init__(self, train_ds, valid_ds, augm_ds, bs=64, device=None, num_workers=4, **kwargs):
        self.device = default_device if device is None else device
        self.train_dl = DeviceDataLoader.create(train_ds, bs,   shuffle=True,  num_workers=num_workers, **kwargs)
        self.valid_dl = DeviceDataLoader.create(valid_ds, bs*2, shuffle=False, num_workers=num_workers, **kwargs)
        self.augm_dl  = DeviceDataLoader.create(augm_ds,  bs*2, shuffle=False, num_workers=num_workers, **kwargs)

    @classmethod
    def create(cls, train_ds, valid_ds, train_tfm=None, valid_tfm=None, dl_tfms=None, **kwargs):
        return cls(DatasetTfm(train_ds, train_tfm), DatasetTfm(valid_ds, valid_tfm), DatasetTfm(valid_ds, train_tfm), 
                   tfms=dl_tfms, **kwargs)

    @property
    def train_ds(self): return self.train_dl.dl.dataset
    @property
    def valid_ds(self): return self.valid_dl.dl.dataset

In [ ]:
tds = transform_datasets(train_ds, valid_ds, tfms, size=size)
data = DataBunch(*tds, num_workers=8, tfms=data_norm)

In [ ]:
learn = ConvLearner(data, arch, 2, wd=1e-3)
learn.metrics = [accuracy]

In [ ]:
learn.load('2')

In [ ]:
_,axs = plt.subplots(2,4,figsize=(12,6))
for ax in axs.flat: show_image(tds[2][1][0], ax)

In [ ]:
model = learn.model
model.eval();

In [ ]:
with torch.no_grad():
    preds,y = zip(*[(model(xb.detach()), yb.detach()) for xb,yb in data.valid_dl])

preds = torch.cat(preds)
y = torch.cat(y)

In [ ]:
accuracy(preds, y)

In [ ]:
def get_preds(model, dl):
    with torch.no_grad():
        return torch.cat([model(xb.detach()) for xb,yb in dl])

In [ ]:
all_preds = torch.stack([get_preds(model, data.augm_dl) for _ in range(4)])

In [ ]:
avg_preds = all_preds.mean(0)
avg_preds.shape

In [ ]:
accuracy(avg_preds, y)

In [ ]:
beta=0.5
accuracy(preds*beta + avg_preds*(1-beta), y)

## Fin